Ce document vise à calculer les probabilités qu'un mot tapé soit un autre mot de notre base de données.
On part du principe qu'on a une liste de mots connus.
On se soucie simplement de regarder la ressemblance avec d'autres mots existants, pas de regarder les mots à proximité.

In [69]:
import numpy as np

In [70]:
dictionnaire = ['aime', 'aimé', 'aimer', 'aimes', 'meme', 'aimez', 'mimez', 'mime', 'même', 'mimer', 'mimes', 'mimé']
mot_tapé = "Amies"
# exemple jouet, on déploiera ça de manière plus générale une fois qu'on aura la data de Quentin

La méthode qu'on va utiliser consiste à regarder le mot tapé de gauche à droite, et de calculer le nombre de lettres différentes pour chaque mot de notre dictionnaire. Puis on attribuera a chaque erreur un score selon son importance (une faute d'accent est moins grave qu'une faute de lettre). Plus le score est élevé, plus le mot sera éloigné du mot auquel on le compare.

Les probabilités calculées le seront a partir d'un score. Il faut considérer l'ensemble des erreurs possibles. On prend :
- La taille du mot qui diffère (infini si trop éloigné, +3 si éloigné de deux caractères, +2 si un caractère)
- Le nombre de lettres différentes (score plus ou moins important si les lettres sont proches de la lettre visée ou pas);
- Eventuelles inversions de lettres (on ne considère que les lettres successives 2 par deux : aime et amie, mais pas oise et soie)
- Fautes d'accent (+1 par faute d'accent)

On commence par mettre le mot en minuscule avant tout traitement. On lui rendra sa majuscule à la fin.


In [71]:
mot_tapé = mot_tapé.lower()
print(mot_tapé)

amies


# Taille du mot

On suppose que deux mots peuvent potentiellement être proches seulement s'ils ont une taille assez similaire (+2 ou -1 caractères)

In [72]:
def is_len_words_near(mot_tapé, mot_dico):
    assert isinstance(mot_tapé, str)
    assert isinstance(mot_dico, str)
    n1 = len(mot_tapé)
    n2 = len(mot_dico)
    return ((n2-2 <= n1) & (n1 <= n2+1))

is_len_words_near('aime', 'aimera')

True

Implémentation du calcul du score :

In [73]:
def score_len(mot_tapé, mot_dico, pénalité_max=2):
    assert isinstance(mot_tapé, str)
    assert isinstance(mot_dico, str)
    n1 = len(mot_tapé)
    n2 = len(mot_dico)
    score = 0
    if not is_len_words_near(mot_tapé, mot_dico):
        score = np.inf
    elif ((n1 == n2-1) | (n1 == n2+1)):
        score += pénalité_max/2
    elif (n1 == n2-2):
        score += pénalité_max
    return score

print(score_len('aime', 'mile'))
print(score_len('aime', 'mille'))
print(score_len( 'aime', 'aimera'))
print(score_len('aime', 'ai'))
print(score_len('aime', 'aie'))


0
1.0
2
inf
1.0


# Première lettre

On suppose aussi que la première lettre du mot est toujours la bonne (c'est ce que font les correcteurs). Donc les mots qui ne commencent pas par la même lettre ont une probabilité nulle aussi.

In [74]:
def is_first_letter_the_same(mot_tapé, mot_dico):
    assert isinstance(mot_tapé, str)
    assert isinstance(mot_dico, str)
    return (mot_tapé[0] == mot_dico[0])

is_first_letter_the_same('aime', 'mimer')

False

Implémentation du score :

In [75]:
def score_prem_lettre(mot_tapé, mot_dico):
    assert isinstance(mot_tapé, str)
    assert isinstance(mot_dico, str)
    if not is_first_letter_the_same(mot_tapé, mot_dico):
        score = np.inf
        return score
    else:
        return 0

# Lettres différentes

Calcul du score pour des lettres différentes : on ajoute 2 au score si la lettre est différente et loin, 1 si elle est différente et proche, 1 si c'est une faute d'accent.

In [86]:
dict_proches_clavier = {"a":["&","é","z","s","q"],"z":["&","é","a","s","q","d","e","\""],"e":["\'","\"","z","s","f","d","r"],
 "t":["-","y","g","f","h","r","("],"y":["-","t","g","j","h","u","è"],"u":["_","y","k","j","h","i","è"],
 "i":["_","u","k","j","l","o","ç"],"o":["à","i","k","m","l","p","ç"],"p":["o","l","m","ù",")","à"],
 "q":["a","z","s","w","x","<"],"s":["a","z","k","q","d","w","x"],"d":["s","z","e","r","f","x","c"],
 "f":["e","r","t","d","g","c","v"],"g":["r","t","y","h","f","v","b"],"h":["t","y","u","g","j","b","n"],
 "j":["u","i","y","h","k","n",","],"k":["u","i","o","j","l",",",";"],"l":["i","o","p","k","m",";",":"],
 "m":["o","p","ù","l",":","!"],"w":["<","q","s","d","x"],"x":["q","s","d","c","w"],"c":["s","d","f","x","v"],
 "v":["f","g","c","b"],"b":["v","g","h","n"],"i":["b","h","j",","],"r":["\'","t","g","f","d","e","("],
 "n":["b","h","j"]}

#création d'un mapping pour vérifier si il y a faute d'accent
mapping_accent_to_lettre = {"é":"e", "è":"e", "ç":"c", "à":"a", "ù":"u", 'â':'a', 'ê':'e', 'î':'i', 'ô':'o', 'û':'u'}
list_accents = ['é', "è","ç","à","ù",'â','ê','î','ô', 'û']

In [77]:
def score_letter_diff(lettre_tapée, lettre_dico, pénalité_max = 2):
    '''Retourne un score de différence entre deux lettres.'''
    assert isinstance(lettre_tapée, str)
    assert len(lettre_tapée) == 1
    assert isinstance(lettre_dico, str)
    assert len(lettre_dico) == 1
    score = 0
    # Part 1 : lettres identiques
    if lettre_tapée == lettre_dico:
        return score
    # Part 2 : fautes d'accents
        # Cas 1 : les 2 lettres ont un accent
    elif (lettre_tapée in (list_accents)) & (lettre_dico in (list_accents)):
        if mapping_accent_to_lettre[lettre_tapée] == mapping_accent_to_lettre[lettre_dico]:
            score += pénalité_max/2
        else:
            score += pénalité_max
        # Cas 2:  1 des 2 lettres a un accent
    elif (lettre_tapée in (list_accents)) & (not (lettre_dico in (list_accents))):
        if mapping_accent_to_lettre[lettre_tapée] == lettre_dico:
            score += pénalité_max/2
        else:
            score += pénalité_max
    elif (lettre_dico in (list_accents)) & (not (lettre_tapée in (list_accents))):
        if mapping_accent_to_lettre[lettre_dico] == lettre_tapée:
            score += pénalité_max/2
        else:
            score += pénalité_max
    
    # Part 3 : lettres proches
    elif lettre_dico in (dict_proches_clavier[lettre_tapée]):
        score += pénalité_max/2
    # Part 4 : rien de tout cela, ie lettres non proches sans faute d'accent
    else:
        score += pénalité_max
    return score
        
    
print(score_letter_diff("c", "ç"))
print(score_letter_diff("c", "v"))
print(score_letter_diff("c", "c"))
print(score_letter_diff("c", "o"))
print(score_letter_diff("m", "a"))


    

1.0
1.0
0
2
2


Reste à itérer sur tout le mot. On normalise pour avoir un score max d'erreurs de mot qui vaut 3 (même chose que si on a 2 lettres de différence entre les deux mots), mais il faudra le faire APRES VERIFICATION DES PERMUTATIONS.

In [78]:
def score_mot_letter_diff(mot_tapé, mot_dico, score_max=3, pénalité_max=2):
    assert isinstance(mot_tapé, str)
    assert isinstance(mot_dico, str)
    n1 = len(mot_tapé)
    n2 = len(mot_dico)
    n = min(n1, n2)
    score = 0
    for i in range (n):
        score += score_letter_diff(mot_tapé[i], mot_dico[i])
    return score*score_max/(pénalité_max*n)
    
print(score_mot_letter_diff("mime", "aimer"))
print(score_mot_letter_diff("mimeras", "aimeras"))
print(score_mot_letter_diff("mimeras", "jobastre"))

0.75
0.42857142857142855
3.0


# Inversions de lettre (WIP)

Cette partie du code consiste à regarder si deux lettres successives du mot tapé sont les mêmes que les deux lettres successives au même endroit du mot dico, à permutation près. Dans ce cas, il faut retirer la pénalité dûe aux deux lettres dans le score précédent de lettres diff.

In [79]:
# normalisation : * score_max / (pénalité_max*min(len(w1), len(w2)))

# Fonction principale

Cette fonction agrège tous les scores calculés précédemment, puis les inverse pour obtenir une "probabilité d'être le mot recherché". On lui ajoute 1 pour être sur d'obtenir un score > 1, utile pour l'inversion qui nous permettra d'obtenir une proba plus petite que 1.

In [90]:
def score_total(mot_tapé, mot_dico):
    return 1 + score_len(mot_tapé, mot_dico) + score_prem_lettre(mot_tapé, mot_dico) + score_mot_letter_diff(mot_tapé, mot_dico)

def proba_mot_proche(mot_tapé, mot_dico):
    return 1/score_total(mot_tapé, mot_dico)

print(proba_mot_proche('habile','babil'))
print(proba_mot_proche('babiller','babil'))
print(proba_mot_proche('habit','babil'))
print(proba_mot_proche('bâtit','babil'))
print(proba_mot_proche('babike','babil'))
print(proba_mot_proche('babio','babil'))
print(proba_mot_proche('babiz','babil'))
print(proba_mot_proche('baby','babil'))
print(proba_mot_proche('bénin', 'babil'))
print(proba_mot_proche('babil', 'babil'))
    

0.0
0.0
0.0
0.4
0.4347826086956522
0.7692307692307692
0.625
0.36363636363636365
0.4
1.0


# Autre méthode : édit distance


In [81]:
def edit_distance(s1, s2):
    # Nombre d'opérations nécessaires pour aller d'un mot à un autre (parmi suppression d'un carac, insertion, substitution)
    dp = [[0 for j in range(len(s2)+1)] for i in range(len(s1)+1)]
    for i in range(1, len(s2)+1):
        dp[0][i] = i
    
    for i in range(1, len(s1)+1):
        dp[i][0] = i

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(min(dp[i-1][j], dp[i][j-1]) + 1, dp[i-1][j-1] + (1 if s1[i-1] != s2[j-1] else 0))
    
    return dp[len(s1)][len(s2)]

In [82]:
def edits1(word):
    """All edits that are one edit away from 'word'"""
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
 
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [83]:
def editsk(word, k):
    """All edits that are k edits away from 'word'"""
    e = edits1(word)
    prev = e
    for i in range(1, k):
        temp = set(e2 for e1 in prev for e2 in edits1(e1))
        e.update(temp)
        prev = temp
    e.discard(word)
    return e